TEXT

In [14]:
# IMPORTS
import random
import numpy as np
import pandas as pd

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

In [15]:
# Set random seed to ensure reproducibility
random.seed(37)

In [16]:
# file_path = 'instances_01_KP/low-dimensional/f1_l-d_kp_10_269'
# file_path = 'instances_01_KP/low-dimensional/f2_l-d_kp_20_878'
file_path = 'instances_01_KP/low-dimensional/f8_l-d_kp_23_10000'

# file_path = 'instances_01_KP/large_scale/knapPI_1_100_1000_1'

data = np.loadtxt(file_path, dtype=int, usecols=(0, 1))
col_1 = data[:, 0]
col_2 = data[:, 1]

n_items = col_1[0]
capacity = col_2[0]

values = data[1:, 0]
weights = data[1:, 1]

# print("First Column:", col_1)
# print("Second Column:", col_2)
print("number of items:", n_items)
print("max weight:", capacity)
print("values:", values)
print("weights:", weights)

number of items: 23
max weight: 10000
values: [981 980 979 978 977 976 487 974 970 485 485 970 970 484 484 976 974 482
 962 961 959 958 857]
weights: [983 982 981 980 979 978 488 976 972 486 486 972 972 485 485 969 966 483
 964 963 961 958 959]


In [17]:
def knapsack_fitness(solution, values, weights, capacity):
    total_weight = np.dot(solution, weights)
    total_value = np.dot(solution, values)
    if total_weight > capacity:
        return 0  # Invalid solution
    return total_value

In [18]:
def generate_random_solution(length):
    return np.random.randint(2, size=length)

In [19]:
# Store problem items in dictionary

items = {}
for i in range(n_items):
    items[i] = (values[i], weights[i])

print(items)
print(len(items))
print(items[3][1])

{0: (981, 983), 1: (980, 982), 2: (979, 981), 3: (978, 980), 4: (977, 979), 5: (976, 978), 6: (487, 488), 7: (974, 976), 8: (970, 972), 9: (485, 486), 10: (485, 486), 11: (970, 972), 12: (970, 972), 13: (484, 485), 14: (484, 485), 15: (976, 969), 16: (974, 966), 17: (482, 483), 18: (962, 964), 19: (961, 963), 20: (959, 961), 21: (958, 958), 22: (857, 959)}
23
980


In [20]:
# Fitness function

def evalIndividual(individual):
    weight = 0
    value = 0
    # print(individual)
    for i in range(n_items):
        value += items[i][0] * individual[i]
        weight += items[i][1] * individual[i]
    if weight > capacity:
        return (0,)
    # print(value)
    return (value,)

In [21]:
def umda_update_full(population, pop_size, select_size, replace_size, toolbox):
    # select from population
    selected_population = tools.selBest(population, select_size)

    # Calculate marginal propabilities
    probabilities = np.mean(selected_population, axis=0)

    new_solutions = []
    for _ in range(pop_size):
        new_solution = np.random.rand(n_items) < probabilities
        new_solution = creator.Individual(new_solution.astype(int).tolist())  # Create as DEAP Individual
        new_solutions.append(new_solution)
    
    return population

def umda_update_partial(population, pop_size, select_size, replace_size, toolbox):
    # select from population
    selected_population = tools.selBest(population, select_size)

    # Calculate marginal propabilities
    probabilities = np.mean(selected_population, axis=0)

    new_solutions = []
    for _ in range(replace_size):
        new_solution = np.random.rand(n_items) < probabilities
        new_solution = creator.Individual(new_solution.astype(int).tolist())  # Create as DEAP Individual
        new_solutions.append(new_solution)
    
    weakest_individuals = tools.selWorst(population, replace_size)
    
    for i in range(replace_size):
        population[population.index(weakest_individuals[i])] = new_solutions[i]
    
    return population

In [22]:
# Create fitness class
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
# weights is positive to signify maximisation of fitness, use negative to minimise
# weights must be a tuple to allow to single and multi objective problems
# to be treated in the same way

# create individual class
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox() # Initialise DEAP toolbox
toolbox.register("attr_int", random.randint, 0, 1) # generate atributes of 0 or 1
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, n=n_items)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("select", tools.selBest)
toolbox.register("evaluate", evalIndividual)

def umda(pop_size, n_generations, select_size, replace_size):
    # initialise population
    population = toolbox.population(pop_size)
    # print(population)

    # determine fitness of initial population
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    for gen in range(n_generations):
        # generate new population
        population = umda_update_full(population, pop_size, select_size, replace_size, toolbox)

        # determine fitness of new population
        fitnesses = list(map(toolbox.evaluate, population))
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
        
        best_ind = tools.selBest(population, 1)[0]

        # print(f'Gen: {gen}, best sol: {best_ind}, best sol fitness: {best_ind.fitness.values[0]}')
        print(f'Gen: {gen}, best sol fitness: {best_ind.fitness.values[0]}')
    
    print('End of Generations')
    


c:\Users\Admin\anaconda3\envs\EA\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Admin\anaconda3\envs\EA\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [24]:
umda(100, 50, 50, 0)

Gen: 0, best sol fitness: 9728.0
Gen: 1, best sol fitness: 9728.0
Gen: 2, best sol fitness: 9728.0
Gen: 3, best sol fitness: 9728.0
Gen: 4, best sol fitness: 9728.0
Gen: 5, best sol fitness: 9728.0
Gen: 6, best sol fitness: 9728.0
Gen: 7, best sol fitness: 9728.0
Gen: 8, best sol fitness: 9728.0
Gen: 9, best sol fitness: 9728.0
Gen: 10, best sol fitness: 9728.0
Gen: 11, best sol fitness: 9728.0
Gen: 12, best sol fitness: 9728.0
Gen: 13, best sol fitness: 9728.0
Gen: 14, best sol fitness: 9728.0
Gen: 15, best sol fitness: 9728.0
Gen: 16, best sol fitness: 9728.0
Gen: 17, best sol fitness: 9728.0
Gen: 18, best sol fitness: 9728.0
Gen: 19, best sol fitness: 9728.0
Gen: 20, best sol fitness: 9728.0
Gen: 21, best sol fitness: 9728.0
Gen: 22, best sol fitness: 9728.0
Gen: 23, best sol fitness: 9728.0
Gen: 24, best sol fitness: 9728.0
Gen: 25, best sol fitness: 9728.0
Gen: 26, best sol fitness: 9728.0
Gen: 27, best sol fitness: 9728.0
Gen: 28, best sol fitness: 9728.0
Gen: 29, best sol fitnes